# Lezione 24 - PCA: Principal Component Analysis

## Sezione 1 - Titolo e obiettivi

Obiettivo: ridurre la dimensionalita mantenendo la varianza rilevante, interpretare le componenti e usarle per visualizzazione o preprocessing ML.

### Cosa impari
- Concetto di componenti principali e varianza spiegata.
- Come scegliere il numero di componenti con soglie e scree plot.
- Come leggere i loadings e collegarli alle feature originali.
- Come usare PCA per visualizzare, comprimere e ridurre rumore.

### Perche serve
Troppe feature portano a overfitting, costi computazionali e difficolta di interpretazione. PCA comprime l'informazione massimizzando la varianza nelle prime componenti e permette visualizzazioni 2D/3D di dati complessi.

### Prerequisiti minimi
- Dati numerici continui (scalati con StandardScaler).
- Conoscenza base di matrice di covarianza e varianza.
- Familiarita con sklearn e pipeline di preprocessing.


## Sezione 2 - Teoria profonda

### 1.1 Perche ridurre la dimensionalita
- Curse of dimensionality: le distanze perdono significato quando le dimensioni crescono.
- Overfitting: troppe feature rispetto ai campioni.
- Visualizzazione: impossibile plottare >3 dimensioni senza riduzione.

### 1.2 Cosa fa PCA (passi)
1. Centra i dati (media zero).
2. Calcola la matrice di covarianza.
3. Estrae autovalori/autovettori (varianza e direzioni).
4. Proietta i dati sulle prime k direzioni (componenti principali).

### 1.3 Varianza spiegata e scelta di k
- Varianza spiegata = quota di informazione catturata da una PC.
- Criteri: soglia cumulativa (es. 90-95%), gomito nello scree plot, Kaiser (autovalore>1 su dati standardizzati).

### 1.4 Loadings e interpretazione
- Loadings = pesi delle feature nella PC (pca.components_).
- Loadings alti in modulo indicano feature che guidano quella componente.
- Segno positivo/negativo indica associazione direzionale.

### 1.5 Quando usare / evitare PCA
- Usa: molte feature correlate, visualizzazione HD, riduzione rumore, velocizzare training.
- Evita: poche feature gia interpretabili, dati categorici, quando serve interpretabilita diretta sulle feature originali.


## Sezione 3 - Schema mentale e decision map

Workflow sintetico dopo scaling:

```
Dati numerici -> StandardScaler -> PCA() esplorativo
            -> Scree plot / soglia varianza -> scegli k o n_components=0.95
            -> PCA finale -> varianza spiegata + loadings
            -> Usa X_pca per visualizzare o addestrare modelli
```

Checklist rapida
- [ ] Dati scalati, nessun NaN.
- [ ] Obiettivo chiaro (visual, compressione, noise reduction, preprocessing).
- [ ] Varianza cumulativa controllata e scelta di k motivata.
- [ ] Loadings letti per interpretare le componenti.
- [ ] Valutato l'impatto su prestazioni (accuracy, tempo) se usata prima di modelli.


## Sezione 4 - Notebook dimostrativo

### Perche questo passo (Demo 1 - PCA Iris)
Mostrare come PCA compatta 4 feature in 2 PC mantenendo quasi tutta la varianza e separando le classi nel piano PC1-PC2.


---

### 1.4 Interpretare i Loadings

I **loadings** ti dicono come ogni feature originale contribuisce alla PC:

```
Esempio: PC1 per dati di clienti

Feature          Loading
─────────────────────────
età              +0.45    ← contribuisce molto (positivo)
reddito          +0.52    ← contribuisce molto (positivo)
spesa_mensile    +0.48    ← contribuisce molto (positivo)
n_figli          -0.10    ← contribuisce poco

Interpretazione: PC1 cattura il "potere d'acquisto"
(età + reddito + spesa vanno insieme)
```

### ️ Importante: Scaling!

**PCA è sensibile alla scala!** Feature con valori grandi dominano.

```
SBAGLIATO:                    CORRETTO:
età: 20-80                    StandardScaler prima di PCA!
reddito: 20000-200000  ← domina tutto
```

---

### 1.5 Quando Usare PCA

|  Usa PCA quando... |  Evita PCA quando... |
|----------------------|------------------------|
| Hai troppe feature (>20) | Le feature sono già poche |
| Vuoi visualizzare dati HD | Ogni feature è importante singolarmente |
| C'è multicollinearità | I dati non sono correlati |
| Vuoi ridurre overfitting | Hai bisogno di interpretabilità diretta |
| Come preprocessing per ML | I dati sono categorici |

###  PCA nel Pipeline ML

```
Dati ─→ StandardScaler ─→ PCA ─→ Modello ML
         (necessario!)    ↓
                      n_components=0.95
                      (mantieni 95% varianza)
```

---

##  2. Schema Mentale

### Workflow PCA

```
┌─────────────────────────────────────────────────────────────────────┐
│                          PCA WORKFLOW                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│  1. PREPARAZIONE                                                    │
│     └── StandardScaler() — OBBLIGATORIO per PCA!                    │
│     └── Verifica: dati numerici, no NaN                             │
│                                                                     │
│  2. PCA ESPLORATIVO (tutte le componenti)                           │
│     └── pca = PCA()  # senza n_components                           │
│     └── pca.fit(X_scaled)                                           │
│     └── Analizza: explained_variance_ratio_                         │
│                                                                     │
│  3. SCEGLI n_components                                             │
│     └── Plot varianza cumulativa                                    │
│     └── Soglia: 90-95% varianza                                     │
│     └── Oppure: PCA(n_components=0.95) automatico                   │
│                                                                     │
│  4. FIT FINALE                                                      │
│     └── pca = PCA(n_components=k)                                   │
│     └── X_pca = pca.fit_transform(X_scaled)                         │
│                                                                     │
│  5. INTERPRETAZIONE                                                 │
│     └── pca.components_ → loadings                                  │
│     └── Quali feature contribuiscono a ogni PC?                     │
│                                                                     │
│  6. USO                                                             │
│     └── Visualizzazione (2D/3D)                                     │
│     └── Input per clustering o classificazione                      │
│     └── Riduzione rumore                                            │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

###  Checklist Pre-PCA

```
□ Dati scalati con StandardScaler?
□ Nessun valore mancante?
□ Solo feature numeriche?
□ Hai un obiettivo chiaro? (visualizzazione, preprocessing, noise reduction)
```

---

##  3. Demo Pratiche

### Demo 1 — Primo PCA: Dataset Iris

Visualizziamo il classico dataset Iris (4 feature) in 2D.

In [ ]:
# Demo 1 - PCA sul dataset Iris (da 4 feature a 2 PC)
# Intento: mostrare quanta varianza resta con 2 componenti e separazione classi.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

iris = load_iris()
X_iris = iris.data
y_iris = iris.target
feature_names = iris.feature_names
assert X_iris.ndim == 2 and not np.isnan(X_iris).any(), "Dati Iris malformati"

# Scaling obbligatorio
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_iris)
assert X_scaled.shape == X_iris.shape

# PCA esplorativa
pca_full = PCA()
pca_full.fit(X_scaled)

# PCA 2 componenti per visualizzazione
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Varianza spiegata da 2 PC: {pca.explained_variance_ratio_.sum()*100:.1f}%")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
colors = ['red', 'green', 'blue']
for i, name in enumerate(iris.target_names):
    mask = y_iris == i
    axes[0].scatter(X_pca[mask, 0], X_pca[mask, 1], c=colors[i], label=name, s=50, alpha=0.7, edgecolors='black')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
axes[0].set_title('Iris in 2D con PCA')
axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].bar(range(1, 5), pca_full.explained_variance_ratio_, alpha=0.7, color='steelblue', label='Varianza singola')
axes[1].plot(range(1, 5), np.cumsum(pca_full.explained_variance_ratio_), 'ro-', label='Cumulativa')
axes[1].axhline(y=0.95, color='red', linestyle='--', alpha=0.6, label='95%')
axes[1].set_xlabel('Componente'); axes[1].set_ylabel('Varianza spiegata'); axes[1].set_title('Scree plot')
axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()


### Perche questo passo (Demo 2 - Interpretare i loadings)
Collegare PC1/PC2 alle feature originali per capire cosa rappresentano le direzioni principali.


In [ ]:
# Demo 2 - Interpretare i loadings
# Intento: collegare PC1/PC2 alle feature originali tramite loadings e biplot.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

assert 'pca' in globals() and 'X_pca' in globals(), "Esegui prima Demo 1"

loadings = pca.components_
df_load = pd.DataFrame(loadings, columns=feature_names, index=['PC1','PC2'])
print("Loadings PC1/PC2:")
print(df_load.round(3))

fig, axes = plt.subplots(1, 2, figsize=(14,5))
im = axes[0].imshow(loadings, cmap='RdBu_r', aspect='auto', vmin=-1, vmax=1)
axes[0].set_xticks(range(len(feature_names)))
axes[0].set_xticklabels([f.replace(' (cm)','') for f in feature_names], rotation=45, ha='right')
axes[0].set_yticks([0,1]); axes[0].set_yticklabels(['PC1','PC2'])
axes[0].set_title('Heatmap loadings')
for i in range(loadings.shape[0]):
    for j in range(loadings.shape[1]):
        axes[0].text(j, i, f"{loadings[i,j]:.2f}", ha='center', va='center', color='white' if abs(loadings[i,j])>0.5 else 'black')
plt.colorbar(im, ax=axes[0], label='Loading')

ax = axes[1]
colors = ['red','green','blue']
for i, name in enumerate(iris.target_names):
    mask = y_iris == i
    ax.scatter(X_pca[mask,0], X_pca[mask,1], c=colors[i], s=30, alpha=0.5, label=name)
scale=3
for i, feat in enumerate(feature_names):
    ax.arrow(0,0, loadings[0,i]*scale, loadings[1,i]*scale, head_width=0.08, head_length=0.08, fc='black', ec='black')
    ax.text(loadings[0,i]*scale*1.15, loadings[1,i]*scale*1.15, feat.replace(' (cm)',''), fontsize=9, ha='center')
ax.set_title('Biplot PC1-PC2 + loadings'); ax.grid(True, alpha=0.3); ax.axhline(0,color='gray',linewidth=0.5); ax.axvline(0,color='gray',linewidth=0.5); ax.legend(loc='lower left')
plt.tight_layout(); plt.show()


### Perche questo passo (Demo 3 - Scelta automatica di n_components)
Usare soglia di varianza e scree plot su un dataset piu grande (digits) per decidere quante componenti tenere.


In [ ]:
# Demo 3 - Scelta automatica di n_components (digits)
# Intento: usare soglie di varianza e scree plot per decidere quante componenti tenere.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits

digits = load_digits()
X_digits = digits.data; y_digits = digits.target
assert X_digits.ndim == 2 and not np.isnan(X_digits).any()

scaler_digits = StandardScaler()
X_digits_scaled = scaler_digits.fit_transform(X_digits)
assert X_digits_scaled.shape == X_digits.shape

soglie = [0.80, 0.90, 0.95, 0.99]
for soglia in soglie:
    p = PCA(n_components=soglia)
    p.fit(X_digits_scaled)
    print(f"Varianza {soglia*100:.0f}% -> {p.n_components_} componenti")

pca_full_digits = PCA()
pca_full_digits.fit(X_digits_scaled)
cumsum = np.cumsum(pca_full_digits.explained_variance_ratio_)

fig, axes = plt.subplots(1,2, figsize=(14,5))
axes[0].plot(range(1,65), pca_full_digits.explained_variance_ratio_, 'b-o', markersize=3)
axes[0].set_title('Scree plot'); axes[0].set_xlabel('Componente'); axes[0].set_ylabel('Varianza spiegata'); axes[0].grid(True, alpha=0.3)
axes[1].plot(range(1,65), cumsum, 'g-o', markersize=3)
axes[1].axhline(y=0.90, color='orange', linestyle='--', label='90%'); axes[1].axhline(y=0.95, color='red', linestyle='--', label='95%')
axes[1].set_title('Varianza cumulativa'); axes[1].set_xlabel('Componente'); axes[1].set_ylabel('Varianza cumulativa'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()


### Perche questo passo (Demo 4 - Visualizzare cifre in PCA 2D)
Vedere come le cifre si distribuiscono in 2D e quanto si separano le classi con sole due componenti.


In [ ]:
# Demo 4 - Distribuzione delle cifre in PCA 2D
# Intento: visualizzare digits in 2D e osservare separazione fra classi.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

assert 'X_digits_scaled' in globals(), "Esegui Demo 3 prima"

pca2 = PCA(n_components=2)
X_digits_2d = pca2.fit_transform(X_digits_scaled)
assert X_digits_2d.shape[1] == 2

fig, axes = plt.subplots(2,5, figsize=(15,6))
axes = axes.flatten()
for digit in range(10):
    ax = axes[digit]
    mask = y_digits == digit
    ax.scatter(X_digits_2d[mask,0], X_digits_2d[mask,1], c='blue', s=10, alpha=0.7)
    ax.set_title(f'Cifra {digit}'); ax.set_xticks([]); ax.set_yticks([]); ax.grid(True, alpha=0.2)
plt.suptitle('Digits in PCA 2D'); plt.tight_layout(); plt.show()


### Perche questo passo (Demo 5 - PCA per denoising)
Usare poche componenti per ricostruire immagini rumorose, osservando il trade-off tra dettaglio e riduzione del rumore.


In [ ]:
# Demo 5 - PCA per riduzione rumore
# Intento: ricostruire immagini rumorose con poche componenti per filtrare il rumore.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

assert 'digits' in globals() and 'X_digits_scaled' in globals(), "Esegui Demo 3 prima"

np.random.seed(42)
indices = [0,100,200,300,400]
X_sample = digits.images[indices].copy()
noise_level = 5
X_noisy = X_sample + np.random.normal(0, noise_level, X_sample.shape)
X_noisy = np.clip(X_noisy, 0, 16)
X_noisy_flat = X_noisy.reshape(len(indices), -1)

n_components_list = [5,10,20,40]
fig, axes = plt.subplots(len(indices), len(n_components_list)+2, figsize=(16,10))

for i, idx in enumerate(indices):
    axes[i,0].imshow(X_sample[i], cmap='gray'); axes[i,0].set_title('Originale' if i==0 else ''); axes[i,0].axis('off')
    axes[i,1].imshow(X_noisy[i], cmap='gray'); axes[i,1].set_title(f'+Rumore' if i==0 else ''); axes[i,1].axis('off')
    for j, n_comp in enumerate(n_components_list):
        pca_dn = PCA(n_components=n_comp)
        pca_dn.fit(X_digits_scaled)
        x_noisy_scaled = (X_noisy_flat[i] - scaler_digits.mean_) / scaler_digits.scale_
        x_pca = pca_dn.transform(x_noisy_scaled.reshape(1,-1))
        x_rec = pca_dn.inverse_transform(x_pca)
        x_rec = x_rec * scaler_digits.scale_ + scaler_digits.mean_
        axes[i, j+2].imshow(x_rec.reshape(8,8), cmap='gray')
        if i==0:
            var = pca_dn.explained_variance_ratio_.sum()*100
            axes[i, j+2].set_title(f'n={n_comp}
({var:.0f}%)')
        axes[i, j+2].axis('off')
for i, idx in enumerate(indices):
    axes[i,0].set_ylabel(f'Cifra {y_digits[idx]}')
plt.suptitle('PCA per denoising'); plt.tight_layout(); plt.show()


## Sezione 5 - Esercizi guidati (step by step)

### Perche questo esercizio (24.1)
Applicare il flusso completo PCA sul dataset Wine: scelta di k, visualizzazione e interpretazione loadings.


In [ ]:
# Esercizio 24.1 - PCA su dataset Wine
# Intento: scegliere componenti per 80/95% varianza, visualizzare e leggere i loadings.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

wine = load_wine()
X_wine = wine.data; y_wine = wine.target; feature_names_wine = wine.feature_names
assert not np.isnan(X_wine).any()

scaler_wine = StandardScaler(); X_wine_scaled = scaler_wine.fit_transform(X_wine)
assert X_wine_scaled.shape == X_wine.shape

pca_wine = PCA(); pca_wine.fit(X_wine_scaled)
cumsum_var = np.cumsum(pca_wine.explained_variance_ratio_)
n_80 = int(np.argmax(cumsum_var>=0.80)+1)
n_95 = int(np.argmax(cumsum_var>=0.95)+1)
print(f"Componenti per 80%: {n_80}, per 95%: {n_95}")

pca_2d = PCA(n_components=2); X_wine_2d = pca_2d.fit_transform(X_wine_scaled)

fig, axes = plt.subplots(1,3, figsize=(16,5))
axes[0].bar(range(1,len(pca_wine.explained_variance_ratio_)+1), pca_wine.explained_variance_ratio_, color='steelblue', alpha=0.7)
axes[0].plot(range(1,len(cumsum_var)+1), cumsum_var, 'ro-', label='Cumulativa')
axes[0].axhline(y=0.80, color='orange', linestyle='--'); axes[0].axhline(y=0.95, color='red', linestyle='--')
axes[0].set_title('Scree plot'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

colors = ['red','green','blue']
for i,name in enumerate(wine.target_names):
    mask = y_wine == i
    axes[1].scatter(X_wine_2d[mask,0], X_wine_2d[mask,1], c=colors[i], s=50, alpha=0.7, edgecolors='black', label=name)
axes[1].set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}%)')
axes[1].set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}%)'); axes[1].set_title('Wine in 2D'); axes[1].legend(); axes[1].grid(True, alpha=0.3)

loadings = pca_2d.components_
axes[2].barh(range(len(feature_names_wine)), loadings[0], alpha=0.7, label='PC1')
axes[2].barh(range(len(feature_names_wine)), loadings[1], alpha=0.7, label='PC2', left=loadings[0])
axes[2].set_yticks(range(len(feature_names_wine))); axes[2].set_yticklabels(feature_names_wine, fontsize=8)
axes[2].set_title('Loadings PC1/PC2'); axes[2].legend(); axes[2].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

# Top feature per interpretazione
import pandas as pd
df_load = pd.DataFrame({'feature':feature_names_wine, 'PC1':loadings[0], 'PC2':loadings[1]})
print(df_load.reindex(df_load['PC1'].abs().sort_values(ascending=False).index).head(3))
print(df_load.reindex(df_load['PC2'].abs().sort_values(ascending=False).index).head(3))


### Perche questo esercizio (24.2)
Valutare l'impatto di PCA sul training di un classificatore confrontando tempo e accuracy prima/dopo la riduzione.


In [ ]:
# Esercizio 24.2 - PCA come preprocessing per classificazione (Digits)
# Intento: confrontare tempo e accuracy con/ senza PCA al 90% di varianza.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_digits

# Dati
if 'digits' not in globals():
    digits = load_digits()
X = digits.data; y = digits.target
assert not np.isnan(X).any()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler_cls = StandardScaler(); X_train_s = scaler_cls.fit_transform(X_train); X_test_s = scaler_cls.transform(X_test)

# Senza PCA
start = time.time(); lr_no = LogisticRegression(max_iter=5000, random_state=42)
lr_no.fit(X_train_s, y_train); t_no = time.time()-start; acc_no = lr_no.score(X_test_s, y_test)

# Con PCA (90%)
pca_cls = PCA(n_components=0.90)
X_train_pca = pca_cls.fit_transform(X_train_s); X_test_pca = pca_cls.transform(X_test_s)
start = time.time(); lr_pca = LogisticRegression(max_iter=5000, random_state=42)
lr_pca.fit(X_train_pca, y_train); t_pca = time.time()-start; acc_pca = lr_pca.score(X_test_pca, y_test)

comparison = pd.DataFrame({
    'Metrica':['Feature','Tempo (s)','Accuracy (%)'],
    'Senza PCA':[X_train_s.shape[1], f"{t_no:.4f}", f"{acc_no*100:.2f}"],
    'Con PCA':[pca_cls.n_components_, f"{t_pca:.4f}", f"{acc_pca*100:.2f}"]
})
print(comparison.to_string(index=False))
feature_reduction = (1 - pca_cls.n_components_/X_train_s.shape[1]) * 100
print(f"Riduzione feature: {feature_reduction:.1f}% | Speedup ~{(t_no/t_pca if t_pca>0 else 1):.2f}x | Delta accuracy {((acc_pca-acc_no)*100):+.2f}%")


### Perche questo esercizio (24.3)
Confrontare separabilita nello spazio originale vs PCA calcolando distanze tra centroidi e visualizzando le classi.


In [ ]:
# Esercizio 24.3 - Confronto separabilita spazio originale vs PCA (Iris)
# Intento: misurare quanto PCA aumenta la distanza tra classi rispetto a due feature originali.

import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris
from scipy.spatial.distance import cdist

iris = load_iris(); X = iris.data; y = iris.target; target_names = iris.target_names
assert not np.isnan(X).any()

X_orig = X[:, :2]  # sepal length, sepal width
scaler_ir = StandardScaler(); X_scaled = scaler_ir.fit_transform(X)
pca_ir = PCA(n_components=2); X_pca = pca_ir.fit_transform(X_scaled)

fig, axes = plt.subplots(1,2, figsize=(14,6))
colors=['red','green','blue']; markers=['o','s','^']
for i,name in enumerate(target_names):
    mask = y==i
    axes[0].scatter(X_orig[mask,0], X_orig[mask,1], c=colors[i], marker=markers[i], s=70, edgecolors='black', alpha=0.7, label=name)
axes[0].set_title('Spazio originale (2 feature)'); axes[0].set_xlabel('Sepal length'); axes[0].set_ylabel('Sepal width'); axes[0].legend(); axes[0].grid(True, alpha=0.3)
for i,name in enumerate(target_names):
    mask = y==i
    axes[1].scatter(X_pca[mask,0], X_pca[mask,1], c=colors[i], marker=markers[i], s=70, edgecolors='black', alpha=0.7, label=name)
axes[1].set_title(f'Spazio PCA (varianza {pca_ir.explained_variance_ratio_.sum()*100:.1f}%)'); axes[1].set_xlabel('PC1'); axes[1].set_ylabel('PC2'); axes[1].legend(); axes[1].grid(True, alpha=0.3)
plt.tight_layout(); plt.show()

def dist_centroidi(Xmat, yvec):
    centroids=[]
    for c in np.unique(yvec):
        centroids.append(Xmat[yvec==c].mean(axis=0))
    centroids=np.array(centroids)
    d=cdist(centroids, centroids)
    vals=d[np.triu_indices_from(d, k=1)]
    return vals.mean()

dist_orig = dist_centroidi(X_orig, y)
dist_pca = dist_centroidi(X_pca, y)
print(f"Distanza media centroidi - originale: {dist_orig:.3f} | PCA: {dist_pca:.3f} | Delta: {(dist_pca-dist_orig)/dist_orig*100:.1f}%")


## Sezione 6 - Conclusione operativa

### Cosa portarsi a casa
- PCA comprime l'informazione ordinando le direzioni per varianza: le prime componenti spiegano quasi tutto se le feature sono correlate.
- Scaling e obbligatorio: senza StandardScaler una feature con range piu grande domina la PCA.
- Scegli n_components con soglie di varianza o scree plot; annota sempre quanta varianza conservi.
- Leggi i loadings per collegare le PC al significato delle feature originali.
- Per classificazione/clustering valuta se la riduzione migliora tempo e separabilita.

### Methods explained (uso, input/output, errori tipici)
- `StandardScaler`: standardizza feature (input array n_samples x n_features, output stessa shape); errori: NaN o colonne costanti.
- `PCA`: calcola componenti principali; input X scalato 2D; output trasformato X_pca e attributi `explained_variance_ratio_`, `components_`; errori: dati non scalati o n_components troppo alto (>n_features).
- `explained_variance_ratio_`: quota di varianza per PC; serve per scree plot e soglia cumulativa.
- `components_` (loadings): righe=PC, colonne=feature; errori: interpretazione senza considerare il segno.
- `StandardScaler` + `PCA` in pipeline: evita leakage fit/transform su train e poi transform su test.

### Common errors and quick debug
- Dati non scalati: PC distorte; fix: applica StandardScaler prima di PCA.
- Troppe poche PC: perdita di informazione e calo accuracy; fix: aumenta soglia varianza e ricontrolla.
- Interpretazione errata dei loadings: confusione su segni e magnitudini; fix: guarda valori assoluti per importanza e il segno per la direzione.
- Applicare PCA a dati categorici: componenti senza senso; fix: usa tecniche per categorie (one-hot + eventualmente altra riduzione come MCA).


## Sezione 7 - End-of-lesson checklist e glossario

### Checklist finale
- [ ] Dati numerici e scalati con `StandardScaler`.
- [ ] Varianza spiegata cumulativa valutata (soglia 90-95%).
- [ ] Numero di componenti scelto e motivato (scree plot/gomito o n_components=valore).
- [ ] Loadings letti per interpretare le PC.
- [ ] Se usata in pipeline ML: verificato impatto su tempo e accuracy.
- [ ] Silhouette/metriche calcolate nel nuovo spazio se usato per clustering.

### Glossario (termini usati)
- Componente Principale (PC): nuova direzione di massima varianza.
- Varianza spiegata: quota di varianza catturata da una PC.
- Scree plot: grafico autovalori/varianza per PC.
- Loadings: pesi delle feature nella PC.
- Kaiser criterion: tieni PC con autovalore>1 (dopo scaling).
- Denoising: riduzione rumore ricostruendo con poche PC.
- n_components: parametro PCA per fissare numero o soglia di varianza.
- StandardScaler: standardizza feature a media 0 e dev. std 1.
- Curse of dimensionality: difficolta dovute ad alto numero di feature.
- Eigenvalue/eigenvector: autovalore/autovettore di covarianza (varianza e direzione).


## Sezione 8 - Didactic changelog (max 10 voci)
1. Riorganizzate le 8 sezioni con obiettivi, teoria, schema mentale, demo ed esercizi guidati.
2. Ripulito testo da emoji/non ASCII e aggiunte rationale per demo/esercizi.
3. Commentati i code cell con scaling, assert e note su silhouette/varianza.
4. Inseriti Methods explained, errori comuni, checklist e glossario nelle sezioni conclusive.
